# The Human Side of PowerShell Scripting

## PowerShell Conference Europe 2025

![Swedish Flag](images/flag.jpg)
### Jeff Hicks

<div style = "background-color:rgb(226, 220, 220); padding-bottom: 25px;padding-top:10px;padding-left: 10px; padding-right: 10px; width:60%">
<img src="images/hicks.png" alt="Jeff Hicks" align="left" vertical-align="top" style="padding-right: 10px ;padding-bottom: 25px;"/>

- 35-year IT Professional
- 19-year Microsoft MVP
- PowerShell Author 
- PowerShell Teacher
- https://jdhitsolutions.github.io
  
### ![MVP Logo](images/mvp-logo.png)

![Sponsors](images/psconfeu-sponsors.png)

## What Are We Talking About?

- Any one can learn PowerShell's syntax and mechanics. Let's focus on the *squishy* in-between bits
- What you should write
- What __not__ to write

<details><summary><b>PowerShell Essential Concepts</b></summary>

<details><summary>Critical</summary>

- PowerShell doesn't exist in a vacuum
<details><summary>AI</summary>

- AI tools are great for generating ___syntax___
<details><summary>PowerShell is not syntax</summary>

- There is more to PowerShell than syntax and mechanics
<details><summary>PowerShell is People</summary>

- __People__ are the most important part of any PowerShell scripting project
- Better PowerShell code is *crafted* not generated.
</details>
<details><summary>Focus</summary>

- __*Don't focus on the specifics of my code samples. Focus on the concepts.*__
</details>

### Command Execution

<details><summary>Execution Details</summary>

- __Who__ will be using your code?
- __What__ are their expectations?
- __Where__ are they executing your code?
- __How__ can you ensure success and make the code as frictionless as possible?
</details>

### Input

- Don't force users to jump through hoops to use your code

In [ ]:
Function Get-DriveInfo {
    [CmdletBinding()]
    Param(
        [Parameter(Position = 0)]
        [string]$DriveLetter = "C:"
    )

    $data = Get-CimInstance -ClassName Win32_Volume -filter "DriveLetter = '$DriveLetter'"
    Write-Host "Processing $($data.Name)" -ForegroundColor Green
    # code continues ...
}

#Sample usage
Get-DriveInfo c
Get-DriveInfo C:\

In [ ]:
#you are forcing them to remember the colon
Get-DriveInfo C:

In [ ]:
# Your code should do the work, not the user
Function Get-DriveInfo {
    [CmdletBinding()]
    Param(
        [Parameter(Position = 0)]
        [ValidatePattern("^[A-Za-z]",ErrorMessage="{0} is an invalid drive letter.")]
        [string]$Drive = "C"
    )

    $DriveLetter = "$($Drive):"
    $data = Get-CimInstance -ClassName Win32_Volume -filter "DriveLetter = '$DriveLetter'"
    if ($data) {
        Write-Host "Processing $($data.Name)" -ForegroundColor Green
        # code continues ...
    } else {
        Write-Warning "Drive $DriveLetter not found."
    }
}

Get-DriveInfo c
Get-DriveInfo Z

In [ ]:
#give the user options with tab-completion
Function Get-DriveInfo {
    [CmdletBinding()]
    Param(
        [Parameter(Position = 0)]
        #add an argument completer for the drive names from win32_LogicalDisk
        [ArgumentCompleter({
            [OutputType([System.Management.Automation.CompletionResult])]
            param(
                [string]$CommandName,
                [string]$ParameterName,
                [string]$WordToComplete,
                [System.Management.Automation.Language.CommandAst] $CommandAst,
                [System.Collections.IDictionary] $FakeBoundParameters
            )

            $CompletionResults = [System.Collections.Generic.List[System.Management.Automation.CompletionResult]]::new()

            $DriveLetters = Get-CimInstance -ClassName Win32_LogicalDisk | Select-Object -ExpandProperty DeviceID
            foreach ($DriveLetter in $DriveLetters) {
                if ($DriveLetter -like "$WordToComplete*") {
                    $CompletionResults.Add((New-Object System.Management.Automation.CompletionResult($DriveLetter, $DriveLetter, 'ParameterValue', $DriveLetter)))
                }
            }

            return $CompletionResults
        })]
        [ValidatePattern("^[A-Za-z]:",ErrorMessage="{0} is an invalid drive letter.")]
        #reverting back because now it is easy for the user to enter valid values
        [string]$Drive = "C:"
    )

    $data = Get-CimInstance -ClassName Win32_Volume -filter "DriveLetter = '$Drive'"
    if ($data) {
        Write-Host "Processing $($data.Name)" -ForegroundColor Green
        # code continues ...
    } else {
        Write-Warning "Drive $DriveLetter not found."
    }
}

# Demo in the live console

### No Assumptions

- Don't assume the user will enter parameter values the same way you do
- Parameter values can come from anywhere
- This is why we have parameter validation and cast to type

In [ ]:
Function Save-Data {
    [cmdletbinding()]
    Param(
        [Parameter(Position = 0,HelpMessage = "Specify the path to save.")]
        [ValidateScript({Test-Path $_ },ErrorMessage = "The path {0} does not exist.")]
        [ValidateNotNullOrEmpty()]
        [string]$Path = ".",

        [ValidateNotNullOrEmpty()]
        [PSCredential]$Credential,

        [switch]$Force
    )

    $cPath = Convert-Path -Path $path
    Write-Host "Saving data from $cPath" -ForegroundColor cyan
    #code that needs a normal file system path
}

In [ ]:
# sample normal execution
$p = Get-ChildItem "c:\scripts\data"
Save-Data -path $p -Force

In [ ]:
# but a user may be trying this
$p = Get-ChildItem c:\scripts\data -directory
Save-Data $p -force

In [ ]:
#or doesn't exist
Save-Data X:\foo

## But try to plan

- What is a likely usage scenario?
- How can your code be easier to use?
- Consider the use of aliases, pipeline input, and parameter sets

In [ ]:
Function Backup-ADComputer {
    [cmdletbinding()]
    Param(
        [Parameter(Position = 0,Mandatory,ValueFromPipelineByPropertyName)]
        [string]$Computername
    )
    Begin {}
    Process {
        Write-Host "Backing up $Computername" -ForegroundColor cyan
        #code
    }
    End {}
}

Backup-ADComputer SRV1

```powershell
Get-ADComputer -Filter * | Backup-ADComputer
```

![AD Fail](images\adcomputer-pipeline-fail.png)


Checking the object properties.

```powershell
PS C:\Users\ArtD> Get-ADComputer SRV1 | Select *


DistinguishedName  : CN=SRV1,CN=Computers,DC=Company,DC=Pri
DNSHostName        : SRV1.Company.Pri
Enabled            : True
Name               : SRV1
ObjectClass        : computer
ObjectGUID         : 12ca78f8-fddb-47dd-9675-30f819174052
SamAccountName     : SRV1$
SID                : S-1-5-21-4162762804-1525231989-1793280658-1104
UserPrincipalName  :
PropertyNames      : {DistinguishedName, DNSHostName, Enabled, Name...}
AddedProperties    : {}
RemovedProperties  : {}
ModifiedProperties : {}
PropertyCount      : 9
```

Normally, I would do this in my function:

```powershell
[Alias("Name")]
[string]$Computername
```

But the AD cmdlets are *weird*. This is one way to make this work with minimal effort.

In [ ]:
Function Backup-ADComputer {
    [cmdletbinding()]
    Param(
        [Parameter(
            Position = 0,
            Mandatory,
            ValueFromPipelineByPropertyName
        )]
        [Alias("ComputerName")]
        [string]$Name
    )
    Begin {}
    Process {
        Write-Host "Backing up $Name" -ForegroundColor cyan
        #code
    }
    End {}
}

```powershell
PS C:\Users\ArtD> Get-ADComputer -Filter "name -like 'srv*'" | Backup-ADComputer
Backing up SRV1
Backing up SRV2
```

However, the alias will still be detected and used.

In [ ]:
Import-CSV c:\scripts\company.csv

In [ ]:
Import-CSV c:\scripts\company.csv | Backup-ADComputer

### Parameters set can also give the user options.

```powershell
Function Backup-ADComputer {
    [cmdletbinding(DefaultParameterSetName = "ByName")]
    Param(
        [Parameter(
            Position = 0,
            ValueFromPipelineByPropertyName,
            ParameterSetName = "ByName"
        )]
        [alias("Name","CN")]
        [ValidateNotNullOrEmpty()]
        [string]$Computername = $env:COMPUTERNAME,
        [Parameter(
            Mandatory,
            ValueFromPipeline,
            ParameterSetName = "ByADComputer"
        )]
        [ValidateNotNullOrEmpty()]
        [Microsoft.ActiveDirectory.Management.ADComputer]$ADComputer
    )
    Begin {}
    Process {
        Write-Verbose "Detected parameter set $($PSCmdlet.ParameterSetName)"
        if ($PSCmdlet.ParameterSetName -eq "byADComputer") {
            $Computername = $ADComputer.Name
        }
        Write-Host "Backing up $Computername" -ForegroundColor cyan
        #code
    }
    End {}
}
```

### Parameter sets exposed in the syntax.

```powershell
PS C:\> Get-Command Backup-ADComputer -Syntax

Backup-ADComputer [[-Computername] <string>] [<CommonParameters>]

Backup-ADComputer -ADComputer <ADComputer> [<CommonParameters>]
```

#### Example

```powershell
PS C:\Users\ArtD> Get-ADComputer -filter "OperatingSystem -like '*server*'" | Backup-ADComputer -Verbose
VERBOSE: Detected parameter set ByADComputer
Backing up DOM1
VERBOSE: Detected parameter set ByADComputer
Backing up SRV1
VERBOSE: Detected parameter set ByADComputer
Backing up SRV2
VERBOSE: Detected parameter set ByADComputer
Backing up DOM2
```

Or using names

```powershell
PS C:\> Import-CSV c:\scripts\company.csv | where class -ne client | Backup-ADComputer -verbose
VERBOSE: Detected parameter set ByName
Backing up Dom1
VERBOSE: Detected parameter set ByName
Backing up Dom2
VERBOSE: Detected parameter set ByName
Backing up Srv1
VERBOSE: Detected parameter set ByName
Backing up Srv2
PS C:\> 
```

## Command Execution

- Give the user meaningful feedback
- Use `Write-Verbose` and `Write-Debug` to provide additional information
- Use `Write-Information` to capture process and data information

In [ ]:
c:\scripts\loadbsky.ps1
Start-BskySession

In [ ]:
Get-BskyProfile -verbose

In [ ]:
start https://github.com/jdhitsolutions/PSBluesky/blob/main/functions/Get-PSBlueSkyProfile.ps1

In [ ]:
# My load script automatically saves Information stream
$iv[0].MessageData

In [ ]:
$n = Get-BskyFeed
$iv[-1].MessageData

## Output and Objects

- It is impossible to know all the ways someone will consume your code
- But try to anticipate likely scenarios
- Write the richest output possible
- Separate formatting from the output

In [ ]:
# Simple function output
Function Resolve-WhoIs {
    [CmdletBinding()]
    Param(
        [Parameter(Position = 0,Mandatory,ValueFromPipeline)]
        [string]$IPAddress
    )
    Begin {
        Write-Verbose "Starting $($MyInvocation.MyCommand)"
        $baseURL = 'http://whois.arin.net/rest'
    }
    Process {
        Write-Verbose "Resolving IP $IPAddress"
        $url = "$baseUrl/ip/$IPAddress"
        $r = Invoke-RestMethod $url
        if ($r.net) {
            [PSCustomObject]@{
                IP                     = $IPAddress
                Name                   = $r.net.name
                RegisteredOrganization = $r.net.orgRef.name
            }
        }
    }
    End {
        Write-Verbose "Ending $($MyInvocation.MyCommand)"
    }
}

Resolve-WhoIs 8.8.8.8

In [ ]:
# Rich function output
Function Resolve-WhoIs {
    [CmdletBinding()]
    Param(
        [Parameter(Position = 0,Mandatory,ValueFromPipeline)]
        [string]$IPAddress
    )
    Begin {
        Write-Verbose "Starting $($MyInvocation.MyCommand)"
        $baseURL = 'http://whois.arin.net/rest'
    }
    Process {
        Write-Verbose "Resolving IP $IPAddress"

        $url = "$baseUrl/ip/$IPAddress"
        $r = Invoke-RestMethod $url
        if ($r.net) {
            $NetBlocks = $r.net.netBlocks.netBlock |
            ForEach-Object { "$($_.StartAddress)/$($_.cidrLength)" }
            $City = (Invoke-RestMethod $r.net.orgRef.'#text').org.city

            [PSCustomObject]@{
                PSTypeName             = 'WhoIsResult'
                IP                     = $IPAddress
                Name                   = $r.net.name
                RegisteredOrganization = $r.net.orgRef.Name
                OrganizationHandle     = $r.net.orgRef.Handle
                City                   = $City
                StartAddress           = $r.net.startAddress
                EndAddress             = $r.net.endAddress
                NetBlocks              = $NetBlocks
                Updated                = $r.net.updateDate -as [DateTime]
            }
        }
    }
    End {
        Write-Verbose "Ending $($MyInvocation.MyCommand)"
    }
}

Resolve-WhoIs 8.8.8.8

### A Complete Example

Here is a working function.

- Follows best practices
- Writes an object to the pipeline

In [ ]:
#requires -version 7.5

Function Get-BasicFileExtensionInfo {
    #this is functioning code but not ideal
    [cmdletbinding()]
    Param(
        [string]$Path = ".",
        [switch]$Recurse,
        [switch]$Hidden
    )

    Begin {
        $enumOpt = [System.IO.EnumerationOptions]::new()
        if ($Recurse) {
            $enumOpt.RecurseSubdirectories = $Recurse
        }
        if ($Hidden) {
            $enumOpt.AttributesToSkip = 2
        }
    }

    Process {
        $dir = Get-Item -Path $Path
        $dir.GetFiles('*', $enumOpt) |
        Group-Object -Property extension -PipelineVariable pv |
        Foreach-Object {
          $_.Group | Measure-Object -Property length -Minimum -Maximum -Average -Sum
        } | Select-Object @{Name="Path";Expression={$Path}},
        @{Name="Extension";Expression={$pv.Name.Replace('.', '')}},Count,
        @{Name="TotalSize";Expression={$_.Sum}},
        @{Name="SmallestSize";Expression={$_.Minimum}},
        @{Name="LargestSize";Expression={$_.Maximum}},
        @{Name="AverageSize";Expression={$_.Average}}
    }
}

Get-BasicFileExtensionInfo c:\temp | Sort TotalSize -descending | Format-Table

### A more human-centered version

- Where have I added value?
- How is this easier to use?
- How is this easier to maintain?

In [ ]:
#requires -version 7.5

#Get-FileExtensionInfo.ps1

using namespace System.Collections.generic

Function Get-FileExtensionInfo {
    #TODO  comment-based help
    [cmdletbinding()]
    [alias('gfei')]
    [OutputType('FileExtensionInfo')]
    Param(
        [Parameter(
            Position = 0,
            ValueFromPipeline,
            HelpMessage = 'Specify the root directory path to search'
        )]
        [ValidateNotNullOrEmpty()]
        [ValidateScript({ Test-Path $_ },ErrorMessage = 'Cannot find or verify the path {0}.')]
        [string]$Path = '.',

        [Parameter(HelpMessage = 'Recurse through all folders.')]
        [switch]$Recurse,

        [Parameter(HelpMessage = 'Include files in hidden folders')]
        [switch]$Hidden,

        [Parameter(HelpMessage = 'Add the corresponding collection of files')]
        [Switch]$IncludeFiles
    )

    Begin {
        Write-Information -MessageData $MyInvocation
        #define a version for this stand-alone function
        $ver = '1.3.0'
        Write-Verbose "[$((Get-Date).TimeOfDay) BEGIN  ] Starting $($MyInvocation.MyCommand) v$ver"
        Write-Verbose "[$((Get-Date).TimeOfDay) BEGIN  ] Running PowerShell version $($PSVersionTable.PSVersion)"
        Write-Verbose "[$((Get-Date).TimeOfDay) BEGIN  ] Using PowerShell Host $($Host.Name)"
        #capture the current date and time for the audit date
        $report = Get-Date

        $enumOpt = [System.IO.EnumerationOptions]::new()
        if ($Recurse) {
            Write-Verbose "[$((Get-Date).TimeOfDay) BEGIN  ] Getting files recursively"
            $enumOpt.RecurseSubdirectories = $Recurse
        }
        if ($Hidden) {
            Write-Verbose "[$((Get-Date).TimeOfDay) BEGIN  ] Including hidden files"
            $enumOpt.AttributesToSkip = 2
        }
        Write-Information -MessageData $enumOpt
        #initialize a list to hold the results
        $list = [list[object]]::new()

        #determine the platform. This will return a value like Linux, MacOS, or Windows
        $platform = (Get-Variable IsWindows, IsMacOS, IsLinux | where { $_.value }).Name -replace 'is', ''
    } #begin

    Process {
        Write-Information -MessageData $PSBoundParameters
        #convert the path to a file system path
        $cPath = Convert-Path -Path $Path

        Write-Verbose "[$((Get-Date).TimeOfDay) PROCESS] Processing $cPath"
        $dir = Get-Item -Path $cPath
        #using the .NET GetFiles() method for performance.
        #the enumOption is not available in Windows PowerShell
        $files = $dir.GetFiles('*', $enumOpt)

        Write-Verbose "[$((Get-Date).TimeOfDay) PROCESS] Getting the total sum of all files"
        $TotalSum = $files | Measure-Object -Property length -Sum
        Write-Information -MessageData $TotalSum
        Write-Verbose "[$((Get-Date).TimeOfDay) PROCESS] Found $($files.count) files"
        $group = $files | Group-Object -Property extension

        #Group and measure
        foreach ($item in $group) {
            Write-Information -MessageData $item.Group
            Write-Verbose "[$((Get-Date).TimeOfDay) PROCESS] Measuring $($item.count) $($item.name) files"
            $measure = $item.Group | Measure-Object -Property length -Minimum -Maximum -Average -Sum

            #create a custom object
            $out = [PSCustomObject]@{
                PSTypeName       = 'FileExtensionInfo'
                Path             = $cPath
                Extension        = $item.Name.Replace('.', '')
                Count            = $item.Count
                PercentTotal     = [math]::Round(($item.Count / $files.Count), 2) #<-- cast as double for sorting
                TotalSize        = $measure.Sum  #<-- don't format numbers here
                TotalSizePercent = [math]::Round(($measure.Sum / $TotalSum.Sum), 4)
                SmallestSize     = $measure.Minimum
                LargestSize      = $measure.Maximum
                AverageSize      = $measure.Average
                Computername     = [System.Environment]::MachineName  #<-- extra information
                Platform         = $platform  #<-- extra information
                ReportDate       = $report  #<-- extra information
                Files            = $IncludeFiles ? $item.group : $null  #<-- extra information
                IsLargest        = $False  #<-- extra information
            }
            $list.Add($out)
        }
    } #process

    End {
        #mark the extension with the largest total size
        ($list | Sort-Object -Property TotalSize, Count)[-1].IsLargest = $True
        #write the results to the pipeline
        $list
        Write-Verbose "[$((Get-Date).TimeOfDay) END    ] Ending $($MyInvocation.MyCommand)"
    } #end
} #close Get-FileExtensionInfo

In [ ]:
$r = Get-FileExtensionInfo c:\presentations -recurse -verbose

In [ ]:
$r | Get-Random -count 1 | Select *

Let's make it even more human-centered.

In [ ]:
#Add an alias property
Update-TypeData -TypeName FileExtensionInfo -MemberType AliasProperty -MemberName Total -Value TotalSize -Force
#Add script properties
Update-TypeData -TypeName FileExtensionInfo -MemberType ScriptProperty -MemberName TotalMB -value { $this.TotalSize/1mb} -Force
Update-TypeData -TypeName FileExtensionInfo -MemberType ScriptProperty -MemberName TotalKB -value { $this.TotalSize/1kb} -Force
#Add a custom format file
Update-FormatData C:\presentations\WorkplaceNinjaUK\Toolmaking\code-samples\FileExtensionInfo.format.ps1xml

### Sample Output
![File Extension Info](images/fileextensioninfo.png)

In [ ]:
$r | Format-Table -view TotalMB

In [ ]:
$r | where IsLargest

In [ ]:
#using the custom alias property
$r | sort total -descending |
Select -first 10 -property Extension,Total,*Size,Path,Computername |
ConvertTo-JSON

In [ ]:
#using the custom script property

$r  | Sort Count -descending | Select Extension,Count,TotalKB -first 10

### Going Further

- The function needs error handling
- Add support for `Write-Progress`
- Property sets
- Additional formatted views
- Think locally, act globally

## TL;DR

<details><summary><b>Summary</b></summary>
<details><summary><i><b>Consider</b></i></summary>

- Consider *who* will be using your PowerShell code and *how* they will be using it.
- *How* might they want to use it in the future or consider edge cases.

<details><Summary><i><b>Don't Assume</b></i></summary>

- __Don't assume__ the user knows what you know or will run your command the way you do.

<details><Summary><i><b>All About the Object</b></i></summary>

- Create rich, *meaningful* objects to the pipeline formatted with <u>maximum</u> information.

<details><Summary><i><b>User First</b></i></summary>

- Write PowerShell tools that are elegant, efficient, effortless for the __user__.
- *__Design__* your command with people in mind. We're not generating code.

<details><Summary><i><b>PowerShell Scripting is a Craft</b></i></summary>

- *__Craft code__* for the next person to maintain, it could be you in six months.
- __Behind the PowerShell Pipeline__ (https://leanpub.com/behind-the-pspipeline)

![Behind the PowerShell Pipeline](images/behind.png)
</details>

## Questions or Comments?

<img src="images\pexels-pixabay-208494.jpg" alt="Q and A" width="45%" height="45%" style="padding-right: 50px ;padding-bottom: 25px; padding-left: 300px"/>

## Thank You!

<img src="images\jeff-hicks.jpg" style="padding-left:275px;" width="45%" height="45%" alt="Jeff and Journey">

### https://jdhitsolutions.github.io